### Import Libraries

In [ ]:
import json
from datetime import datetime
from collections import defaultdict
import pandas as pd

### Load JSON data

In [ ]:
file_path = 'insurance-policies.json'
with open(file_path, 'r') as file:
    data = json.load(file)

### Initialize dictionaries for aggregating data

In [ ]:
monthly_sales = defaultdict(int)  # Count of sales per month
cancellations_per_month = defaultdict(int)  # Count of cancellations per month
policies_starting_per_month = defaultdict(int)  # Count of policies starting per month
monthly_premium_collections = defaultdict(float)  # Total premium collected per month
monthly_ipt_contributions = defaultdict(float)  # Total IPT contributions per month
monthly_commissions = defaultdict(float)  # SERL's commission earnings per month
products_revenue = defaultdict(lambda: defaultdict(float))  # Revenue per product per month

### Function to parse dates, handling different formats

In [ ]:
def parse_date(date_str):
    if date_str:
        try:
            # Attempt to parse datetime with time component
            return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        except ValueError:
            # Fallback to date-only format
            return datetime.strptime(date_str, "%Y-%m-%d")
    return None

### Function to calculate IPT given a premium, assuming a fixed IPT rate

In [ ]:
def calculate_ipt(premium, ipt_rate=0.12):
    return premium * ipt_rate

### Process each policy in the dataset

In [ ]:
for policy in data:
    # Handle both 'sale_date' and 'sale_dates' keys if they exist
    sale_date_str = policy.get('sale_date') or policy.get('sale_dates')
    sale_date = parse_date(sale_date_str)
    
    start_date = parse_date(policy['start_date'])
    cancel_date = parse_date(policy['cancel_date'])
    premium = policy['premium']
    commission_SERL_percent = policy['commission_SERL_percent']
    product_name = policy['product_name']

    # Aggregate sales data
    if sale_date:
        sale_month = sale_date.strftime("%Y-%m")
        monthly_sales[sale_month] += 1
        monthly_premium_collections[sale_month] += premium
        monthly_ipt_contributions[sale_month] += calculate_ipt(premium)
        monthly_commissions[sale_month] += premium * (commission_SERL_percent / 100)
        products_revenue[sale_month][product_name] += premium * (commission_SERL_percent / 100)
    
    # Aggregate cancellation data
    if cancel_date:
        cancel_month = cancel_date.strftime("%Y-%m")
        cancellations_per_month[cancel_month] += 1
    
    # Aggregate policy start data
    if start_date:
        start_month = start_date.strftime("%Y-%m")
        policies_starting_per_month[start_month] += 1

# Calculate month-over-month sales growth
months_sorted = sorted(monthly_sales.keys())
mom_sales_growth = {months_sorted[i]: ((monthly_sales[months_sorted[i]] / monthly_sales[months_sorted[i-1]] - 1) * 100 if i > 0 else 0) for i in range(len(months_sorted))}

# Identify top-performing products per month
top_performing_products = {month: max(products, key=products.get) for month, products in products_revenue.items()}

### Create a DataFrame from the aggregated data

In [7]:
df = pd.DataFrame({
    'Month': months_sorted,
    'Sales': [monthly_sales[month] for month in months_sorted],
    'Cancellations': [cancellations_per_month.get(month, 0) for month in months_sorted],
    'Policies Starting': [policies_starting_per_month.get(month, 0) for month in months_sorted],
    'MoM Sales Growth (%)': [mom_sales_growth[month] for month in months_sorted],
    'Premium Collections': [monthly_premium_collections[month] for month in months_sorted],
    'IPT Contributions': [monthly_ipt_contributions[month] for month in months_sorted],
    'SERL Commissions': [monthly_commissions[month] for month in months_sorted],
    'Top Performing Product': [top_performing_products[month] for month in months_sorted]
})

# Convert 'Month' column to datetime to sort chronologically, then back to string
df['Month'] = pd.to_datetime(df['Month'])
df.sort_values('Month', inplace=True)
df['Month'] = df['Month'].dt.strftime('%Y-%m')

df

,Month,Sales,Cancellations,Policies Starting,MoM Sales Growth (%),Premium Collections,IPT Contributions,SERL Commissions,Top Performing Product
0,2019-01,7,0,6,0.000000,3934.0,472.08,164.26,Life Insurance
1,2019-02,6,0,6,-14.285714,3612.0,433.44,114.00,Pet Insurance
2,2019-03,4,0,5,-33.333333,2479.0,297.48,144.08,Pet Insurance
3,2019-04,5,0,2,25.000000,3381.0,405.72,136.49,Life Insurance
4,2019-05,6,0,8,20.000000,1531.0,183.72,48.68,Home Insurance
...,...,...,...,...,...,...,...,...,...
67,2024-08,14,3,13,75.000000,7715.0,925.80,327.13,Life Insurance
68,2024-09,9,0,12,-35.714286,5788.0,694.56,285.23,Pet Insurance
69,2024-10,8,3,8,-11.111111,5257.0,630.84,164.73,Pet Insurance
70,2024-11,5,5,6,-37.500000,3224.0,386.88,149.42,Life Insurance


### Save the DataFrame to a CSV file

In [ ]:
csv_file_path = 'insurance_policies_analysis.csv'
df.to_csv(csv_file_path, index=False)